# Lab 7: Structured Output - Respostas em Formato Definido

<img src="./assets/LC_StructuredOutput.png" width="500">

## 🎯 O que você vai aprender neste Lab

**Structured Output** permite que agentes produzam dados em formatos definidos (JSON, objetos Python). Isso é essencial para integrar agentes com sistemas existentes.

### Por que Structured Output é importante?

1. **Integração com sistemas**: APIs esperam dados em formatos específicos
2. **Validação automática**: Dados são validados contra o schema
3. **Processamento programático**: Fácil de processar em código
4. **Consistência**: Sempre o mesmo formato de resposta

### O Problema com Texto Livre:

```python
# Texto livre - difícil de processar
resposta = "O nome é João, email joao@email.com, telefone 11999999999"

# Como extrair os dados? Regex? Parse manual? 😰
```

### Com Structured Output:

```python
# Formato estruturado - fácil de usar
resposta = {
    "name": "João",
    "email": "joao@email.com",
    "phone": "11999999999"
}

# Acesso direto!
print(resposta["email"])  # joao@email.com
```

### Tipos suportados pelo LangChain:

| Tipo | Descrição | Validação |
|------|-----------|-----------|
| **TypedDict** | Dicionário tipado | Tipos básicos |
| **Pydantic BaseModel** | Modelo com validação | Completa |
| **dataclass** | Classe de dados Python | Tipos básicos |
| **JSON Schema (dict)** | Schema JSON puro | Estrutura |

## ⚙️ Setup - Configuração do Ambiente

Instalação das bibliotecas necessárias.

In [ ]:
%pip install -U mlflow>=3 langchain>=1 langchain-community databricks-langchain --quiet
dbutils.library.restartPython()

In [ ]:
import mlflow
mlflow.langchain.autolog()

## 📋 Exemplo: Extração de Contatos com TypedDict

Vamos criar um agente que extrai informações de contato de uma conversa gravada.

### Definindo o schema com TypedDict

`TypedDict` permite definir a estrutura esperada da resposta:

In [ ]:
from typing_extensions import TypedDict
from langchain.agents import create_agent
from databricks_langchain import ChatDatabricks


class ContactInfo(TypedDict):
    name: str
    email: str
    phone: str


# Inicializar o modelo Databricks
llm = ChatDatabricks(
    endpoint="databricks-meta-llama-3-3-70b-instruct",
    temperature=0.1,
)

agent = create_agent(model=llm, response_format=ContactInfo)

recorded_conversation = """We talked with John Doe. He works over at Example. His number is, let's see, 
five, five, five, one two three, four, five, six seven. Did you get that?
And, his email was john at example.com. He wanted to order 50 boxes of Captain Crunch."""

result = agent.invoke(
    {"messages": [{"role": "user", "content": recorded_conversation}]}
)

result["structured_response"]

### O que acontece:

1. Definimos `ContactInfo` com os campos esperados
2. Passamos `response_format=ContactInfo` ao criar o agente
3. O agente extrai os dados da conversa e retorna em `result["structured_response"]`

Observe como o agente interpreta "five, five, five..." como um número de telefone!

## 🔷 Alternativa: Pydantic BaseModel

**Pydantic** oferece validação mais robusta que `TypedDict`. Você pode adicionar:
- Validadores customizados
- Valores padrão
- Descrições de campos
- Restrições (min/max, regex, etc.)

### Vantagens do Pydantic:
- Validação automática de tipos
- Serialização/deserialização JSON
- Documentação automática (OpenAPI)
- Mensagens de erro claras

* pydantic `BaseModel`
* `TypedDict`
* `dataclasses`
* json schema (dict)

### Resultado com Pydantic

O resultado agora é uma instância de `ContactInfoPydantic` com validação de tipos!

In [ ]:
from langchain.agents import create_agent
from pydantic import BaseModel


class ContactInfoPydantic(BaseModel):
    name: str
    email: str
    phone: str


agent = create_agent(model=llm, response_format=ContactInfoPydantic)

recorded_conversation = """ We talked with John Doe. He works over at Example. His number is, let's see, 
five, five, five, one two three, four, five, six seven. Did you get that?
And, his email was john at example.com. He wanted to order 50 boxes of Captain Crunch."""

result = agent.invoke(
    {"messages": [{"role": "user", "content": recorded_conversation}]}
)

result["structured_response"]

## 📚 Resumo e Próximos Passos

### O que aprendemos:

| Formato | Quando usar |
|---------|-------------|
| **TypedDict** | Estruturas simples, sem validação complexa |
| **Pydantic BaseModel** | Validação robusta, APIs de produção |
| **dataclass** | Estruturas simples com métodos |
| **JSON Schema** | Integração com sistemas externos |

### Como acessar a resposta estruturada:

```python
result = agent.invoke(...)

# A resposta estruturada está em:
dados = result["structured_response"]

# Acesso aos campos:
nome = dados["name"]      # TypedDict
nome = dados.name         # Pydantic
```

### Dicas:
- Use Pydantic para validação em produção
- Adicione descrições aos campos para ajudar o LLM
- Considere campos opcionais (`Optional[str]`) quando apropriado

### Próximo Lab:
No **Lab 8 - Dynamic Prompts**, você aprenderá a adaptar o prompt do agente dinamicamente com base em contexto!